# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
#WORKING

# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,15.52,61,5,5.87,AU,1699660288
1,1,puerto ayora,-0.7393,-90.3518,24.19,92,55,4.47,EC,1699660139
2,2,isafjordur,66.0755,-23.1240,1.12,80,99,1.83,IS,1699660003
3,3,bridgewater,44.3835,-64.5155,6.82,72,99,3.60,CA,1699660289
4,4,montagu,-40.7667,144.9667,14.74,57,86,8.69,AU,1699660289


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["City", "Humidity"],
    alpha=0.4
)

# Display the map
city_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

#define ideal weather criteria
min_temp = 24
max_temp =30
max_humidity = 50
max_cloudiness = 25
max_wind_speed = 10

#filter the df
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= min_temp) & 
    (city_data_df["Max Temp"] <= max_temp) & 
    (city_data_df["Humidity"] < max_humidity) & 
    (city_data_df["Cloudiness"] < max_cloudiness) & 
    (city_data_df["Wind Speed"] < max_wind_speed)
]

# Drop any rows with null values
ideal_weather_df_cleaned = city_data_df.dropna()

# Display sample data
ideal_weather_df_cleaned.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,15.52,61,5,5.87,AU,1699660288
1,1,puerto ayora,-0.7393,-90.3518,24.19,92,55,4.47,EC,1699660139
2,2,isafjordur,66.0755,-23.1240,1.12,80,99,1.83,IS,1699660003
3,3,bridgewater,44.3835,-64.5155,6.82,72,99,3.60,CA,1699660289
4,4,montagu,-40.7667,144.9667,14.74,57,86,8.69,AU,1699660289


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
219,revolucion mexicana,MX,26.3110,-108.9435,44,
298,karratha,AU,-20.7377,116.8463,37,
424,barra do garcas,BR,-15.8900,-52.2567,23,
442,alice springs,AU,-23.7000,133.8833,35,
444,aleg,MR,17.0481,-13.9209,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel

radius = 10000
limit = 1
params = {
    'limit':limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"category=accommodation.hotel&circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
revolucion mexicana - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
barra do garcas - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
aleg - nearest hotel: No hotel found
tera - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
219,revolucion mexicana,MX,26.3110,-108.9435,44,No hotel found
298,karratha,AU,-20.7377,116.8463,37,No hotel found
424,barra do garcas,BR,-15.8900,-52.2567,23,No hotel found
442,alice springs,AU,-23.7000,133.8833,35,No hotel found
444,aleg,MR,17.0481,-13.9209,14,No hotel found
527,tera,NE,14.0078,0.7531,10,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
city_hotel_map = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600, 
    size='Humidity', 
    scale=2, 
    color='City', 
    alpha=.4,
    hover_cols=['City', 'Hotel Name', 'Country', 'Humidity']  # Add 'Hotel Name' and 'Country' to hover tooltip
)

# Display the map
city_hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)